In [5]:
print('ok')

ok


In [7]:
import os
os.chdir('../')

In [8]:
%pwd


'c:\\Users\\Sneha Sumedh\\Documents\\Sumedh\\Projects\\Medical-Chatbot-Gen-AI'

In [33]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

In [24]:
# Extract data from pdf file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return  documents

In [25]:
extracted_data = load_pdf_file(data='Data/')

In [27]:
#split the data into smaller chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [28]:
text_chunks = text_split(extracted_data)
print(f"Number of text chunks: {len(text_chunks)}")

Number of text chunks: 5859


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [51]:
# Download the embeddings model
def download_embeddings_model():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [52]:
embeddings = download_embeddings_model()

In [56]:
from dotenv import load_dotenv
load_dotenv()

True

In [57]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [48]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = 'medicalbot'

pc.create_index(
    name=index_name,
    dimension=384,
    metric='cosine',
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1",
    )
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-pw0pptv.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [58]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
# Embed each chunk and upsert embeddings into your pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks, 
    embedding=embeddings, 
    index_name=index_name
    )

In [ ]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [63]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [60]:
from langchain_openai import OpenAI
llm = OpenAI(temperature =0.4,max_tokens=500)

In [61]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question answering tasks."
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, just say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
      
    ]

)

In [64]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [65]:
response = rag_chain.invoke({"input": "What are the symptoms of diabetes?"})
print(response["answer"])



Symptoms of diabetes include fatigue and high levels of glucose in the blood (hyperglycemia). If left untreated, it can lead to damage or failure of various organs in the body. Hypoglycemia, or low blood sugar, may also be discovered through blood sugar testing.
